# Digit Recgonizer

## Introduction

In this notebook, we will build and train a Convolutional Neural Network (CNN) for the task of handwritten digit recognition using the famous MNIST dataset. The goal is to achieve high accuracy in classifying handwritten digits from 0 to 9.

We will go through various steps of the machine learning pipeline, including data loading, data visualization, data preprocessing, model building, training, evaluation, and prediction.

Let's get started!

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import the necessary libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping

## Exploary Data Analysis (EDA)

In [ ]:
# Load the data
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
# Size of the datasets
print(f"Train Data Shape ==> {train_data.shape}")
print(f"Test Data Shape  ==> {test_data.shape}")

In [ ]:
train_data.head()

In [ ]:
# Define the label counts
label_counts = train_data['label'].value_counts()
print(label_counts)

In [ ]:
# Create an interactive vertical bar chart using Plotly
fig = px.bar(y=label_counts.values, x=label_counts.index, labels={'x': 'Label', 'y': 'Count'},
             text=label_counts.values, title='Label Distribution in the Training Data', orientation='v')
fig.update_traces(texttemplate='%{text}', textposition='outside', marker_color=px.colors.qualitative.Plotly)
fig.update_layout(xaxis_title='Label', yaxis_title='Count', showlegend=False)
fig.show()

In [ ]:
# Visualize some digits
plt.figure(figsize=(12, 6))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(train_data.iloc[i, 1:].values.reshape(28, 28), cmap='gray')
    plt.title(f"Label: {train_data.iloc[i, 0]}", pad=12)
    plt.axis('off')
plt.tight_layout()
plt.show()

## Data Preprocessing

In [ ]:
# Split the data into features and labels
X = train_data.drop('label', axis=1).values.astype('float32')
y = train_data['label'].values

# Normalize the pixel values to [0, 1]
X /= 255.0

# Reshape the data to 28x28x1 (height, width, channels)
X = X.reshape(-1, 28, 28, 1)

# Convert labels to one-hot encoded vectors
y = tf.keras.utils.to_categorical(y, num_classes=10)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
print(f"X_train shape ==> {X_train.shape}")
print(f"X_val shape   ==> {X_val.shape}")
print(f"y_train shape ==> {y_train.shape}")
print(f"y_val shape   ==> {y_val.shape}")

## Building and Training the Model (Convolutional Neural Network - CNN)

In [ ]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Data Augmentation

In [ ]:
# Create a data generator with random transformations
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest'
)

# Fit the data generator on the training data
datagen.fit(X_train)

# Use the data generator during training
history = model.fit(datagen.flow(X_train, y_train, batch_size=128), epochs=10, validation_data=(X_val, y_val))

### Learning Rate Scheduling

In [ ]:
# Define a learning rate schedule function
def lr_schedule(epoch):
    initial_lr = 0.001
    if epoch < 5:
        return initial_lr
    else:
        return initial_lr * tf.math.exp(0.1 * (5 - epoch))

# Use the learning rate schedule during training
lr_scheduler = LearningRateScheduler(lr_schedule)
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_val, y_val), callbacks=[lr_scheduler])

### Regularization (Dropout)

In [ ]:
# Create the CNN model with dropout layers
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Add dropout here
model.add(Dense(10, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Early Stopping

In [ ]:
# Use early stopping during training
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, batch_size=128, epochs=50, validation_data=(X_val, y_val), callbacks=[early_stopping])

## Model Evaluation

In [ ]:
# Smmary of the model
model.summary()

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
# Plot learning curves
plt.figure(figsize=(6, 4))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Create the confusion matrix
y_pred_val = np.argmax(model.predict(X_val), axis=1)
cm = confusion_matrix(np.argmax(y_val, axis=1), y_pred_val)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

### Misclassified Examples

In [ ]:
# Find misclassified examples
misclassified_idx = np.where(y_pred_val != np.argmax(y_val, axis=1))[0]

# Count the number of misclassified examples
num_misclassified = len(misclassified_idx)

# Print the count
print(f"Number of Misclassified Examples: {num_misclassified}")

In [ ]:
# Plot some misclassified examples
plt.figure(figsize=(12, 6))
for i, idx in enumerate(misclassified_idx[:10]):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_val[idx].reshape(28, 28), cmap='gray')
    plt.title(f"True: {np.argmax(y_val[idx])}, Pred: {y_pred_val[idx]}", pad=12)
    plt.axis('off')
plt.tight_layout()
plt.show()

## Making Predictions and Generating Submission File

In [ ]:
# Preprocess test data
X_test = test_data.values.astype('float32')
X_test /= 255.0
X_test = X_test.reshape(-1, 28, 28, 1)

# Make predictions
predictions = model.predict(X_test)
y_pred = np.argmax(predictions, axis=1)

In [ ]:
print(f"X_test shape ==> {X_test.shape}")
print(f"y_pred shape ==> {y_pred.shape}")

In [ ]:
# Save the entire model to a file
model.save('trained_model.keras')

### Displaying Some Predicted Images

In [ ]:
# Randomly select a few examples from the test set
num_examples_to_display = 10
random_indices = np.random.choice(len(X_test), num_examples_to_display, replace=False)
selected_images = X_test[random_indices]
selected_labels_true = y_pred[random_indices]

# Display the selected images along with their predicted labels
plt.figure(figsize=(12, 6))
for i in range(num_examples_to_display):
    plt.subplot(2, 5, i + 1)
    plt.imshow(selected_images[i].reshape(28, 28), cmap='gray')
    plt.title(f"Predicted: {selected_labels_true[i]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Create submission file
submission = pd.DataFrame({'ImageId': np.arange(1, len(y_pred)+1), 'Label': y_pred})
submission.to_csv('submission.csv', index=False)

In [ ]:
def predict_user_image(file_path):
    try:
        # Load and preprocess the user-provided image
        user_image = Image.open(file_path).convert('L')  # Convert to grayscale
        user_image = user_image.resize((28, 28))  # Resize to 28x28 pixels
        user_image = np.array(user_image)  # Convert to NumPy array

        # Invert pixel values to get black background and white number
        user_image = 255 - user_image

        user_image = user_image.astype('float32') / 255.0  # Normalize (assuming you used this preprocessing before)
        user_image = user_image.reshape(1, 28, 28, 1)  # Reshape to match the model's input shape

        # Make predictions using the trained model
        user_prediction = model.predict(user_image)
        user_label = np.argmax(user_prediction)

        # Display the user-provided image and the predicted label
        plt.imshow(user_image.reshape(28, 28), cmap='gray')
        plt.title(f"Predicted: {user_label}")
        plt.axis('off')
        plt.show()

    except Exception as e:
        print("Error: ", e)

In [ ]:
predict_user_image("/kaggle/input/images/0.jpg")

In [ ]:
predict_user_image("/kaggle/input/images/1.jpg")

In [ ]:
predict_user_image("/kaggle/input/images/2.jpg")

In [ ]:
predict_user_image("/kaggle/input/images/3.jpg")

In [ ]:
predict_user_image("/kaggle/input/images/4.jpg")

In [ ]:
predict_user_image("/kaggle/input/images/5.jpg")

In [ ]:
predict_user_image("/kaggle/input/images/6.jpg")

In [ ]:
predict_user_image("/kaggle/input/images/7.jpg")

In [ ]:
predict_user_image("/kaggle/input/images/8.jpg")

In [ ]:
predict_user_image("/kaggle/input/images/9.jpg")

# Conclusion

- In this project, we successfully built and trained a CNN model for handwritten digit recognition. We performed data augmentation, implemented learning rate scheduling, applied dropout regularization, and used early stopping to prevent overfitting.

- Our trained model achieved impressive accuracy on the validation set and was able to accurately predict digits from user-provided images as well.

- We also analyzed misclassified examples and visualized the model's performance using confusion matrices and learning curves.

- Overall, this project demonstrates the power of deep learning and CNNs in solving image classification tasks.


Thank you for following along and I hope to upvote it.

Made by: **Abdelrahman Eldaba**